In [6]:
import cvxpy as cp
import numpy as np
import csv

In [7]:
#import the resource codes
def load_file_60d(filename):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        data={}
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[1]!='1':
                break
            data[next_line[2]] = 'UNK'
#             data.append({next_line[2]:'UNK'})
    return data  

In [8]:
#map resource code to resource type
def code_to_type(filename, data):
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break
            if next_line[2] in data:
                if data[next_line[2]]!='UNK':
                    break
                data[next_line[2]] = next_line[3]

In [109]:
#add the resource types together
def wind_solar_added(filename, data, wind, solar, percent_wind , percent_solar):
    wind_total = 20271
    solar_total = 1008
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
                if next_line[0] not in (wind and solar):
                    solar[next_line[0]] = 0
                    wind[next_line[0]] = 0
                    percent_wind[next_line[0]] = 0
                if (data[next_line[2]])=='WIND':
                    wind[next_line[0]] = (float(wind[next_line[0]]) + float(next_line[3]))
                    percent_wind[next_line[0]] = (float(wind[next_line[0]]) + float(next_line[3]))/wind_total
                if (data[next_line[2]])=='PVGR':
                    solar[next_line[0]]= float(solar[next_line[0]]) + float(next_line[3])
                    percent_solar[next_line[0]] = (float(solar[next_line[0]]) + float(next_line[3]))/solar_total
#     print (solar)

In [154]:
#get the total amount of solar and wind per region
def map_to_regions(filename, solar_data, wind_data, wind_regions, solar_regions):
    jan_march_wind=0
    apr_sep_wind=0
    oct_dec_wind=0
    jan_march_solar=0
    apr_sep_solar=0
    oct_dec_solar = 0

    
    with open(filename) as file:
            reader = csv.reader(file)
            next(reader)
            while True:
                try:
                    next_line = next(reader)
                except StopIteration:
                    break
                if (next_line[0] == 'WIND-C') or (next_line[0] == 'WIND'):
                    jan_march_wind+= float(next_line[2])
                    apr_sep_wind+= float(next_line[3])
                    oct_dec_wind+= float(next_line[4])
                    wind_regions[next_line[1]] += float(next_line[3])
                if next_line[0] == 'SOLAR':
                    jan_march_solar+= float(next_line[2])
                    apr_sep_solar+= float(next_line[3])
                    oct_dec_solar+= float(next_line[4])
                    solar_regions[next_line[1]] += float(next_line[3])

    for i in wind_regions:
        wind_regions[i] = float(wind_regions[i]/apr_sep_wind)
    for i in solar_regions:
        solar_regions[i] = float(solar_regions[i]/apr_sep_solar)
    
    for i in wind_data:
        wind_regions_copy = dict(wind_regions)
        for j in wind_regions_copy:
            wind_regions_copy[j] = float(wind_regions_copy[j]*wind_data[i])
        wind_data[i] = wind_regions_copy
            
    for i in solar_data:
        solar_regions_copy = dict(solar_regions)
        for j in solar_regions_copy:
            solar_regions_copy[j] = float(solar_regions_copy[j]*solar_data[i])
        solar_data[i] = solar_regions_copy
        

In [155]:
data = load_file_60d('60d_SCED_SMNE_GEN_RES-17-APR-18.csv')
code_to_type('60d_SCED_Gen_Resource_Data-17-APR-18.csv', data)

wind = {}
solar = {}
percent_wind = {}
percent_solar = {}
wind_solar_added('60d_SCED_SMNE_GEN_RES-17-APR-18.csv',data, wind, solar, percent_wind, percent_solar)


wind_regions = {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
solar_regions= {'Coast':0, 'East':0, 'FarWest':0, 'North':0, 'NorthCentral':0, 'Northwest':0, 'South':0, 
                    'SouthCentral':0,'West':0}
map_to_regions('zonal_generator_capacities_wo_biomass.csv',solar, wind, wind_regions, solar_regions)

print(wind)

{'02/16/2018 00:14:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 459.91305666110424, 'North': 442.5268610173908, 'NorthCentral': 230.89302130433526, 'Northwest': 447.3043324198788, 'South': 531.7787131275077, 'SouthCentral': 0.0, 'West': 638.8510764633824}, '02/16/2018 00:29:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 450.4351783511342, 'North': 433.40727705066195, 'NorthCentral': 226.13478292243096, 'Northwest': 438.08629442596737, 'South': 520.8198289256859, 'SouthCentral': 0.0, 'West': 625.6856473171188}, '02/16/2018 00:44:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 438.252663452706, 'North': 421.68530047439884, 'NorthCentral': 220.01871895937654, 'Northwest': 426.2377686776495, 'South': 506.73368372603653, 'SouthCentral': 0.0, 'West': 608.7632907017314}, '02/16/2018 00:59:59': {'Coast': 0.0, 'East': 0.0, 'FarWest': 425.75280758799784, 'North': 409.65797944303455, 'NorthCentral': 213.74333833108125, 'Northwest': 414.0806020090198, 'South': 492.2806101075767, 'SouthCentral': 0.0, 'West